In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq

In [ ]:
flipkart_url = "https://www.flipkart.com/search?q=" + "iphone14"

In [ ]:
flipkart_url

In [ ]:
uClient = uReq(flipkart_url)

In [ ]:
flipkart_page = uClient.read()

In [ ]:
flipkart_html = bs(flipkart_page, "html.parser")

In [ ]:
bigboxes = flipkart_html.findAll("div", {"class":"_1AtVbE col-12-12"})

In [ ]:
len(flipkart_html.findAll("div", {"class":"_1AtVbE col-12-12"}))

In [ ]:
box = bigboxes[4]

In [ ]:
box.div.div.div.a['href']

In [ ]:
product_link = "https://www.flipkart.com" + box.div.div.div.a['href']

In [ ]:
prodRes = requests.get(product_link)

In [ ]:
prodRes.encoding = "utf-8"

In [ ]:
prod_html = bs(prodRes.text, 'html.parser')

In [ ]:
commentboxes = prod_html.findAll("div", {"class": "_16PBlm"})

## Ratings

In [ ]:
for i in range(len(commentboxes)-1):
    print((commentboxes[i].div.div.div.findAll("div", {"class": "_3LWZlK _1BLPMq"})[0].text))

## Title

This will provide comment tags: Fabulous!,
Terrific,
Good choice,
Best in the market!,
Must buy!,
Fabulous!,
Perfect product!,
Classy product,
Fabulous!,
Highly recommended

In [ ]:
for i in range(len(commentboxes)-1):
    print(commentboxes[i].div.div.div.findAll("p", {"class": "_2-N8zT"})[0].text)

## Names

In [ ]:
for i in range(len(commentboxes)-1):
	print(commentboxes[i].div.div.findAll("p", {"class": "_2sc7ZR _2V5EHH"})[0].text)

## Reviews

In [ ]:
for i in range(len(commentboxes)-1):
    comtag = commentboxes[i].div.div.findAll("div", {"class": ""})
    print(comtag[0].div.text,"\n")

#### Now we will iterate through all the review pages with the help of for loop

In [ ]:
reviews = []
for i in range(1, 999):
    url = f'https://www.flipkart.com/apple-iphone-14-starlight-128-gb/product-reviews/itm3485a56f6e676?pid=MOBGHWFHABH3G73H&lid=LSTMOBGHWFHABH3G73HVXY5AV&marketplace=FLIPKART&page={i}'
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    
    commentboxes = prod_html.findAll("div", {"class": "_1AtVbE col-12-12"})
    
    for commentbox in commentboxes:
        try:
            name = commentbox.div.div.findAll("p", {"class": "_2sc7ZR _2V5EHH"})[0].text
        except:
            name = "No name"

        try:
            rating = commentbox.div.div.div.div.text
        except:
            rating = "No rating"

        try:
            commentHead = commentbox.div.div.div.p.text
        except:
            commentHead = "No Comment Head"
            
        try:
            comtag = commentbox.div.div.findAll("div", {"class": ''})
            custComment = comtag[0].div.text
        except Exception as e:
            print("Exception while creating dictionary: ", e)

        mydict = {"Name": name, "Rating": rating, "CommentHead": commentHead, "Comment": custComment }
        reviews.append(mydict)


In [ ]:
len(reviews)

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(reviews)
csv = df.to_excel('iphone14reviewss.xlsx', index=False)

### ----------------------------------End of Day1


### Start of Day2

In [1]:
import pandas as pd
import numpy as np

In [9]:
df=pd.read_excel("iphone14reviewss.xlsx")
df.head()

,Name,Rating,CommentHead,Comment
0,No name,NaN,No Comment Head,NaN
1,No name,No rating,No Comment Head,NaN
2,No name,No rating,No Comment Head,NaN
3,No name,4.6★,No Comment Head,NaN
4,Rahul Prasad,5Fabulous!,Fabulous!,Best smart phone under this price range compar...


In [10]:
df.dropna(inplace=True)

In [12]:
df.shape

(1067, 4)

In [13]:
df=df.drop(df[df['Name'] == 'No name'].index).reset_index()

In [14]:
df.shape

(961, 5)

In [15]:
df

,index,Name,Rating,CommentHead,Comment
0,4,Rahul Prasad,5Fabulous!,Fabulous!,Best smart phone under this price range compar...
1,5,Sankhadip Mazumder,4Good choice,Good choice,It's a apple device .. that means you get the ...
2,6,Flipkart Customer,5Terrific,Terrific,Damn such a great phone. Camera is really good...
3,7,Flipkart Customer,5Best in the market!,Best in the market!,"Excellent smart phone, Good battery backup and..."
4,8,Feecon Behera,5Must buy!,Must buy!,Just awesome!!I switch to iPhone 14 from iPhon...
...,...,...,...,...,...
956,1572,Rohit Gupta,4Really Nice,Really Nice,Very beautiful
957,1573,Flipkart Customer,5Classy product,Classy product,Great 👍
958,1579,Anuj Setia,5Must buy!,Must buy!,Very good product and delivered in one day...
959,1580,Agrim Kalas,5Fabulous!,Fabulous!,jeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee...


In [16]:
#will clean off the CommentHead from Rating column and just extract the number rating number
df['Rating'] = df['Rating'].str.split('\D').str[0]

In [17]:
print(len(df['Comment'].unique()))
print(len(df['Comment']))

735
961


We have 961 total comments for now whereas 735 unique comments
The agenda over here is to do the customer sentiment analysis where we won't need NAME column. Whereas, for Flipkart it might be very much useful to do analysis on based of name though.

In [18]:
df=df.drop('index',axis=1)

In [20]:
df.index = range(1, len(df)+1)

In [22]:
df=df.drop('Name',axis=1)

Now, we will try to find the word count and will form a new column besides it to do further analysis and also to know if the longer reviews pay any effect(whether positive or negative)

In [23]:
df['Word count'] = df['Comment'].apply(lambda x: len(x.split()))

In [25]:
df['Char count'] = df['Comment'].apply(lambda x: len(x))

In [26]:
df.head()

,Rating,CommentHead,Comment,Word count,Char count
1,5,Fabulous!,Best smart phone under this price range compar...,58,322
2,4,Good choice,It's a apple device .. that means you get the ...,104,500
3,5,Terrific,Damn such a great phone. Camera is really good...,72,384
4,5,Best in the market!,"Excellent smart phone, Good battery backup and...",14,93
5,5,Must buy!,Just awesome!!I switch to iPhone 14 from iPhon...,42,239


Okay, so we have two columns which will count the no. of words and other will count the total no. of characters of particular Comment. But over here, I just though that in my further analysis the blank spaces will mess with the sentiment analysis. So, for futher cleaning, I can just make a new column which will have the total character counts and ignore the blank spaces.

In [27]:
df['Comment Length'] = df['Comment'].apply(lambda x: len(x.replace(" ",""))if x else 0)

In [28]:
df.head()

,Rating,CommentHead,Comment,Word count,Char count,Comment Length
1,5,Fabulous!,Best smart phone under this price range compar...,58,322,265
2,4,Good choice,It's a apple device .. that means you get the ...,104,500,397
3,5,Terrific,Damn such a great phone. Camera is really good...,72,384,313
4,5,Best in the market!,"Excellent smart phone, Good battery backup and...",14,93,80
5,5,Must buy!,Just awesome!!I switch to iPhone 14 from iPhon...,42,239,198


In [29]:
#We don't need the char count column now! Let's drop it
df = df.drop('Char count',axis=1)

In [30]:
def average_words(x):
    words = x.split()
    return sum(len(word) for word in words) / len(words)

In [31]:
df['Avg Word Length'] = df['Comment'].apply(lambda x: average_words(x))

In [34]:
df.sample(10)

,Rating,CommentHead,Comment,Word count,Comment Length,Avg Word Length
732,5,Fantastic phone ! Great value at Flipkart !,I like the new iphone 14. the battery is defin...,43,191,4.441860
170,5,Awesome,Great product at this price range i am very ha...,14,56,4.000000
614,5,Great product,So far so good,4,11,2.750000
607,5,Mind-blowing purchase,good,1,4,4.000000
505,5,Classy product,good,1,4,4.000000
529,3,Does the job,Not value for money 🤑......,5,23,4.600000
28,5,Best in the market!,Excellent phone... But not usefull for all. If...,23,107,4.652174
662,5,Mind-blowing purchase,Very Nice Video(no need any gimbal) and Photo...,11,58,5.272727
357,4,Pretty good,Great product and performance,4,26,6.500000
116,1,Don't waste your money,Battery performance too low,4,24,6.000000


### Text preprocessing/cleaning with NLTK

Now for further analysis we will find how many stopwords each comment contains. Stopwords are the words which have no informative text.

In [35]:
from nltk.corpus import stopwords

In [36]:
stop_words = stopwords.words('english')

In [37]:
df['Stopword_Count']=df['Comment'].apply(lambda x: len([word for word in x.split() if word.lower() in stop_words]))

In [38]:
df['Stopword Rate'] = df['Stopword_Count'] / df['Word count']

In [39]:
df.sort_values(by='Stopword Rate')

,Rating,CommentHead,Comment,Word count,Comment Length,Avg Word Length,Stopword_Count,Stopword Rate
961,5,Mind-blowing purchase,Awesome,1,7,7.000000,0,0.000000
658,5,Wonderful,Excellent mobile,2,15,7.500000,0,0.000000
373,5,Worth every penny,Good,1,4,4.000000,0,0.000000
372,5,Best in the market!,Matchless,1,9,9.000000,0,0.000000
371,5,Excellent,Matchless,1,9,9.000000,0,0.000000
...,...,...,...,...,...,...,...,...
52,5,Classy product,What a phone.,3,11,3.666667,2,0.666667
412,5,Great product,I am happy with the product,6,22,3.666667,4,0.666667
769,5,Mind-blowing purchase,Is all about this selfie ❤️,6,22,3.666667,4,0.666667
400,5,Simply awesome,i love it,3,7,2.333333,2,0.666667


### Cleaning to make it lower and then split to find remove stopwords

#### First of all we will convert the numerical data into correwsponding words. For eg.: 6 will be converted to six

In [40]:
import inflect 
q = inflect.engine() 
def convert_num(text): 
    # split strings into list of texts 
    temp_string = text.split() 
    # initialise empty list 
    new_str = [] 
  
    for word in temp_string: 
        # if text is a digit, convert the digit 
        # to numbers and append into the new_str list 
        if word.isdigit(): 
            temp = q.number_to_words(word) 
            new_str.append(temp) 
  
        # append the texts as it is 
        else: 
            new_str.append(word) 
  
    # join the texts of new_str to form a string 
    temp_str = ' '.join(new_str) 
    return temp_str 

In [41]:
df['Comment'] = df['Comment'].apply(lambda x: convert_num(x))

#### And now we remove punctuations!

In [42]:
df['Comment'] = df['Comment'].str.replace('[^\w\s]','')

C:\Users\UDIT\AppData\Local\Temp\ipykernel_9984\307403981.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Comment'] = df['Comment'].str.replace('[^\w\s]','')


##### So, uptil now our comment column has no punctuations and numbers are converted to text with inflect engine

So, as the stopwords are not adding value; I will make a new column which will contain the modified comment words which are not in stopwords.

In [44]:
df['Excluding stopwords'] = df['Comment'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

In [46]:
df.sample(5)

,Rating,CommentHead,Comment,Word count,Comment Length,Avg Word Length,Stopword_Count,Stopword Rate,Excluding stopwords
673,4,Nice product,Nice product,2,11,5.500000,0,0.000000,Nice product
770,5,Classy product,Very nice phone Ordered midnight Black though ...,14,70,5.000000,4,0.285714,Very nice phone Ordered midnight Black though ...
937,5,Must buy!,Product is great but worst packaging,6,31,5.166667,2,0.333333,Product great worst packaging
746,5,Simply awesome,Costly but handy and good looking,6,28,4.666667,2,0.333333,Costly handy good looking
790,5,Perfect product!,Awesome,1,7,7.000000,0,0.000000,Awesome


In [47]:
pd.Series(" ".join(df['Excluding stopwords']).split()).value_counts()[:60]

phone          188
good           134
product        133
Nice           106
Good           100
I               96
camera          77
iPhone          68
Awesome         68
quality         60
Very            57
fourteen        55
performance     54
Best            54
battery         51
Flipkart        39
Excellent       38
nice            37
awesome         37
best            37
iphone          36
Great           35
Camera          33
amazing         33
delivery        31
one             30
mobile          30
great           28
Product         28
Amazing         27
apple           25
love            24
happy           24
price           24
Battery         24
The             24
display         23
experience      23
Just            23
time            22
Phone           22
go              22
Apple           21
thirteen        20
Super           19
money           19
flipkart        19
Its             18
day             17
life            17
really          16
first           16
using       

these are the words which are recurring n no. of times and also these are the words which are recurring and also not included in the stopwords list of nltk


In [48]:
other_stopwords = ['product', 'I', 'The', 'Just', 'go', 'Its', 'get', 'My', 'first']
len(other_stopwords)

9

In [50]:
df['Cleaned Comments'] = df['Excluding stopwords'].apply(lambda x: " ".join(word for word in x.split() if word not in other_stopwords))

In [51]:
df.sample(5)

,Rating,CommentHead,Comment,Word count,Comment Length,Avg Word Length,Stopword_Count,Stopword Rate,Excluding stopwords,Cleaned Comments
124,5,Awesome,Nice,1,4,4.000000,0,0.000000,Nice,Nice
42,5,Just wow!,Great product However the package box in which...,27,128,4.740741,9,0.333333,Great product However package box phone arrive...,Great However package box phone arrived upto m...
861,5,Best in the market!,Very nice product i am very,7,23,3.285714,4,0.571429,Very nice product,Very nice
411,5,Terrific,Needless to say iPhone fourteen best in its se...,15,56,3.733333,6,0.400000,Needless say iPhone fourteen best segment Got ...,Needless say iPhone fourteen best segment Got ...
48,5,Classy product,Very attractive Must buyLove It,5,31,6.200000,1,0.200000,Very attractive Must buyLove It,Very attractive Must buyLove It


In [54]:
#tryn to scrap more stopwords
pd.Series(" ".join(df['Cleaned Comments']).split()).value_counts()[:60]

phone          188
good           134
Nice           106
Good           100
camera          77
iPhone          68
Awesome         68
quality         60
Very            57
fourteen        55
performance     54
Best            54
battery         51
Flipkart        39
Excellent       38
best            37
awesome         37
nice            37
iphone          36
Great           35
amazing         33
Camera          33
delivery        31
one             30
mobile          30
Product         28
great           28
Amazing         27
apple           25
Battery         24
price           24
happy           24
love            24
display         23
experience      23
time            22
Phone           22
Apple           21
thirteen        20
flipkart        19
money           19
Super           19
day             17
life            17
using           16
really          16
low             15
everything      15
like            15
better          15
service         14
No              14
buy         

## Lemmatization

As stemming, lemmatization do the same but the only difference is that lemmatization ensures that root word belongs to the language. Because of the use of lemmatization we will get the valid words. In NLTK(Natural language Toolkit), we use WordLemmatizer to get the lemmas of words. We also need to provide a context for the lemmatization.So, we added pos(parts-of-speech) as a paramete

In [58]:
from textblob import Word

In [61]:
df['Lemmatized Comments'] = df['Cleaned Comments'].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))

In [62]:
df.sample(7)

,Rating,CommentHead,Comment,Word count,Comment Length,Avg Word Length,Stopword_Count,Stopword Rate,Excluding stopwords,Cleaned Comments,Lemmatized Comments
590,5,Perfect product!,Gud flipcart,2,11,5.500000,0,0.000000,Gud flipcart,Gud flipcart,Gud flipcart
457,5,Brilliant,What you can say negative about iphone except ...,15,67,4.466667,7,0.466667,What say negative iphone except costly Rest ev...,What say negative iphone except costly Rest ev...,What say negative iphone except costly Rest ev...
807,5,Simply awesome,No need to review apple Fantastic product,7,35,5.000000,2,0.285714,No need review apple Fantastic product,No need review apple Fantastic,No need review apple Fantastic
458,5,Terrific,Nice,1,4,4.000000,0,0.000000,Nice,Nice,Nice
507,5,Mind-blowing purchase,Fantastic,1,9,9.000000,0,0.000000,Fantastic,Fantastic,Fantastic
505,5,Classy product,good,1,4,4.000000,0,0.000000,good,good,good
87,5,Must buy!,After a losts of efforts when you get then Jus...,12,46,3.833333,7,0.583333,After losts efforts get Just Awesome,After losts efforts Awesome,After lost effort Awesome


## Sentiment Analysis

In [64]:
from textblob import TextBlob

In [72]:
#Polarity is what we are more interested in as it gives an sheer idea of how positve(+1) and how negative(-1) the reviews are
#Range of polarity will be -1 to +1 as discussed above
df['Polarity'] = df['Lemmatized Comments'].apply(lambda x: TextBlob(x).sentiment[0])

In [69]:
#Subjectivity is a metric between 0 and 1, and a measure of how much a review is based on factual information vs generic opinion
df['Subjectivity'] = df['Lemmatized Comments'].apply(lambda x: TextBlob(x).sentiment[1])

In [70]:
df.sample(10)

,Rating,CommentHead,Comment,Word count,Comment Length,Avg Word Length,Stopword_Count,Stopword Rate,Excluding stopwords,Cleaned Comments,Lemmatized Comments,Polarity,Subjectivity
678,4,Good quality product,Good i phone,3,10,3.333333,1,0.333333,Good phone,Good phone,Good phone,0.70000,0.600000
916,5,Classy product,Ok,1,2,2.000000,0,0.000000,Ok,Ok,Ok,0.50000,0.500000
105,5,Simply awesome,Best Phone in market,4,18,4.500000,1,0.250000,Best Phone market,Best Phone market,Best Phone market,1.00000,0.300000
5,5,Must buy!,Just awesomeI switch to iPhone fourteen from i...,42,198,4.714286,13,0.309524,Just awesomeI switch iPhone fourteen iPhone 12...,awesomeI switch iPhone fourteen iPhone 12 powe...,awesomeI switch iPhone fourteen iPhone 12 powe...,0.43125,0.479167
787,5,Classy product,Excellent,1,9,9.000000,0,0.000000,Excellent,Excellent,Excellent,1.00000,1.000000
595,5,Great product,Good and genuine product,4,24,6.000000,1,0.250000,Good genuine product,Good genuine,Good genuine,0.55000,0.550000
503,5,Perfect product!,Very,1,4,4.000000,1,1.000000,Very,Very,Very,0.20000,0.300000
437,5,Terrific purchase,Best phone ever,3,13,4.333333,0,0.000000,Best phone ever,Best phone ever,Best phone ever,1.00000,0.300000
918,3,Nice,Camera quality is not good compared to price,8,37,4.625000,3,0.375000,Camera quality good compared price,Camera quality good compared price,Camera quality good compared price,0.70000,0.600000
676,5,Brilliant,Good,1,4,4.000000,0,0.000000,Good,Good,Good,0.70000,0.600000


In [71]:
df.describe()

,Word count,Comment Length,Avg Word Length,Stopword_Count,Stopword Rate,Polarity,Subjectivity
count,961.000000,961.000000,961.000000,961.000000,961.000000,961.000000,961.000000
mean,7.963580,37.829344,5.022871,2.616025,0.194181,0.547756,0.652250
std,12.887483,58.952930,1.633151,5.456200,0.204305,0.344783,0.316143
min,1.000000,2.000000,2.000000,0.000000,0.000000,-1.000000,0.000000
25%,2.000000,8.000000,4.000000,0.000000,0.000000,0.333333,0.500000
50%,3.000000,16.000000,4.714286,1.000000,0.187500,0.600000,0.666667
75%,8.000000,41.000000,5.600000,3.000000,0.357143,0.780000,1.000000
max,104.000000,412.000000,30.000000,57.000000,1.000000,1.000000,1.000000


## My Conclusion:

Based on the mean polarity and subjectivity values that you obtained from the sentiment analysis of 1000 reviews using TextBlob, we can make some conclusions about the overall sentiment of the reviews.

The mean polarity value of 0.547756 indicates that the sentiment of the reviews is slightly positive on average. Polarity ranges from -1 to +1, with -1 being very negative and +1 being very positive. A polarity value of 0 indicates a neutral sentiment. Therefore, a mean polarity value of 0.547756 suggests that the reviews are more positive than neutral, but not overwhelmingly positive.

The mean subjectivity value of 0.652250 indicates that the reviews are moderately subjective on average. Subjectivity ranges from 0 to 1, with 0 being very objective and 1 being very subjective. A subjectivity value of 0.5 indicates a neutral subjective score. Therefore, a mean subjectivity value of 0.652250 suggests that the reviews are mostly based on personal opinions and feelings, but there is still some level of objectivity in them.

Overall, based on these results, we can conclude that the 1000 reviews analyzed using TextBlob are slightly positive on average and moderately subjective. However, it's important to keep in mind that sentiment analysis using TextBlob or any other tool is not perfect and may not always accurately reflect the true sentiment of the text. It's always a good idea to manually review a sample of the reviews to ensure that the sentiment analysis is consistent with the actual sentiment of the text.


## Learning Outcome:

Unsupervised Sentiment Analysis with TextBlob helped me learn about Polarity and Subjectivity scores in depth and also helped dig more into how the Natural Language Processing Toolkit can do wonders with text data.

Also, here there was no need for labelling the data for unsupervised learning model as the output itself was the sentiment score(in the form of tuple(Polarity, Subjectivity)).These techniques use lexicons and heuristics to determine the sentiment of the text data.